In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

ds_train, ds_info = tfds.load(
    'forest_fires',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
# Keep numeric columns
X = []
for s in a[1].keys():
    if a[1][s].dtype != tf.string:
        X.append(a[1][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[0].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[0].numpy()[test_idx]

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = np.inf

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = wlgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['rmse'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['regression']),
    'metric':hp.choice('metric',['rmse']),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.5663798998904135, 'btype': 'R', 'extra_dims': 14, 'feature_fraction': 0.9277117511512822, 'lambda_l1': 0.005403325991990691, 'lambda_l2': 0.04192369341689001, 'learning_rate': 0.45276785346755777, 'metric': 'rmse', 'min_data_in_leaf': 65.0, 'min_sum_hessian_in_leaf': 3.663033166166908, 'num_leaves': 806.0, 'objective': 'regression'}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `fobj` in train.
Moving param `metric` to an feval.                     
[1]	valid_0's rmse: 95.7762                            
[2]	valid_0's rmse: 369.834                            
[3]	valid_0's rmse: 1855.35                            
[4]	valid_0's rmse: 9741.21                            
[5]	valid_0's rmse: 50997.2                            
[6]	valid_0's rmse: 269335                             
[7]	valid_0's rmse: 1.42914e+06                        
[8]	valid_0's rmse: 7.57661e+06                        
[9]	valid_0's rmse: 4

[18]	valid_0's rmse: 68.7722                                                    
[19]	valid_0's rmse: 68.7722                                                    
[20]	valid_0's rmse: 68.7722                                                    
[21]	valid_0's rmse: 68.7722                                                    
[22]	valid_0's rmse: 68.7722                                                    
[23]	valid_0's rmse: 68.7722                                                    
[24]	valid_0's rmse: 68.7722                                                    
[25]	valid_0's rmse: 68.7722                                                    
[26]	valid_0's rmse: 68.7722                                                    
[27]	valid_0's rmse: 68.7722                                                    
[28]	valid_0's rmse: 68.7722                                                    
[29]	valid_0's rmse: 68.7722                                                    
[30]	valid_0's rmse: 68.7722

[10]	valid_0's rmse: 68.3361                                                    
[11]	valid_0's rmse: 68.2977                                                    
[12]	valid_0's rmse: 68.2602                                                    
[13]	valid_0's rmse: 68.2236                                                    
[14]	valid_0's rmse: 68.1877                                                    
[15]	valid_0's rmse: 68.1515                                                    
[16]	valid_0's rmse: 68.1172                                                    
[17]	valid_0's rmse: 68.0825                                                    
[18]	valid_0's rmse: 68.0498                                                    
[19]	valid_0's rmse: 68.0166                                                    
[20]	valid_0's rmse: 67.985                                                     
[21]	valid_0's rmse: 67.9532                                                    
[22]	valid_0's rmse: 67.9234

[2]	valid_0's rmse: 298.182                                                     
[3]	valid_0's rmse: 1243.83                                                     
[4]	valid_0's rmse: 5111.87                                                     
[5]	valid_0's rmse: 22893.8                                                     
[6]	valid_0's rmse: 99881.5                                                     
[7]	valid_0's rmse: 439546                                                      
[8]	valid_0's rmse: 1.95687e+06                                                 
[9]	valid_0's rmse: 8.77144e+06                                                 
[10]	valid_0's rmse: 3.87538e+07                                                
[11]	valid_0's rmse: 1.7296e+08                                                 
[12]	valid_0's rmse: 7.71504e+08                                                
[13]	valid_0's rmse: 3.40667e+09                                                
[14]	valid_0's rmse: 1.52637

Overwriting param `num_class`                                                   
Overwriting param `objective` while setting `fobj` in train.                    
Moving param `metric` to an feval.                                              
[1]	valid_0's rmse: 68.5115                                                     
[2]	valid_0's rmse: 68.3252                                                     
[3]	valid_0's rmse: 68.2229                                                     
[4]	valid_0's rmse: 68.1933                                                     
[5]	valid_0's rmse: 68.1614                                                     
[6]	valid_0's rmse: 68.3815                                                     
[7]	valid_0's rmse: 68.6256                                                     
[8]	valid_0's rmse: 68.6536                                                     
[9]	valid_0's rmse: 68.8645                                                     
[10]	valid_0's rmse: 69.1336

...

[87]	valid_0's rmse: 68.5136                                                    
[88]	valid_0's rmse: 68.5509                                                    
[89]	valid_0's rmse: 68.5996                                                    
[90]	valid_0's rmse: 68.5201                                                    
[91]	valid_0's rmse: 68.4879                                                    
[92]	valid_0's rmse: 68.5384                                                    
[93]	valid_0's rmse: 68.533                                                     
[94]	valid_0's rmse: 68.6102                                                    
[95]	valid_0's rmse: 68.5775                                                    
[96]	valid_0's rmse: 68.526                                                     
[97]	valid_0's rmse: 68.5047                                                    
[98]	valid_0's rmse: 68.5342                                                    
[99]	valid_0's rmse: 68.6185

[80]	valid_0's rmse: 50859.1                                                    
[81]	valid_0's rmse: 111908                                                     
[82]	valid_0's rmse: 251190                                                     
[83]	valid_0's rmse: 560729                                                     
[84]	valid_0's rmse: 1.25529e+06                                                
[85]	valid_0's rmse: 2.80541e+06                                                
[86]	valid_0's rmse: 6.27638e+06                                                
[87]	valid_0's rmse: 1.40353e+07                                                
[88]	valid_0's rmse: 2.57206e+07                                                
[89]	valid_0's rmse: 4.71266e+07                                                
[90]	valid_0's rmse: 1.00557e+07                                                
[91]	valid_0's rmse: 1.43842e+07                                                
[92]	valid_0's rmse: 2.04976

[73]	valid_0's rmse: 68.7722                                                    
[74]	valid_0's rmse: 68.7722                                                    
[75]	valid_0's rmse: 68.7722                                                    
[76]	valid_0's rmse: 68.7722                                                    
[77]	valid_0's rmse: 68.7722                                                    
[78]	valid_0's rmse: 68.7722                                                    
[79]	valid_0's rmse: 68.7722                                                    
[80]	valid_0's rmse: 68.7722                                                    
[81]	valid_0's rmse: 68.7722                                                    
[82]	valid_0's rmse: 68.7722                                                    
[83]	valid_0's rmse: 68.7722                                                    
[84]	valid_0's rmse: 68.7722                                                    
[85]	valid_0's rmse: 68.7722

[66]	valid_0's rmse: 71.2308                                                    
[67]	valid_0's rmse: 71.2308                                                    
[68]	valid_0's rmse: 71.2308                                                    
[69]	valid_0's rmse: 71.2308                                                    
[70]	valid_0's rmse: 71.2308                                                    
[71]	valid_0's rmse: 71.2308                                                    
[72]	valid_0's rmse: 71.2308                                                    
[73]	valid_0's rmse: 71.2308                                                    
[74]	valid_0's rmse: 71.2308                                                    
[75]	valid_0's rmse: 71.2308                                                    
[76]	valid_0's rmse: 71.2308                                                    
[77]	valid_0's rmse: 71.2308                                                    
[78]	valid_0's rmse: 71.2308

[59]	valid_0's rmse: 68.3677                                                    
[60]	valid_0's rmse: 68.2778                                                    
[61]	valid_0's rmse: 68.296                                                     
[62]	valid_0's rmse: 68.437                                                     
[63]	valid_0's rmse: 68.5666                                                    
[64]	valid_0's rmse: 68.6421                                                    
[65]	valid_0's rmse: 68.6638                                                    
[66]	valid_0's rmse: 68.6956                                                    
[67]	valid_0's rmse: 68.7101                                                    
[68]	valid_0's rmse: 68.8962                                                    
[69]	valid_0's rmse: 68.9797                                                    
[70]	valid_0's rmse: 69.0302                                                    
[71]	valid_0's rmse: 69.0576

In [4]:
print(best_val)
print(space_eval(spc, best))

65.2899276851524
{'bagging_fraction': 0.6717168979670439, 'btype': 'In', 'extra_dims': 10, 'feature_fraction': 0.7466331446375792, 'lambda_l1': 0, 'lambda_l2': 7.385966055147798, 'learning_rate': 0.9430705524333947, 'metric': 'rmse', 'min_data_in_leaf': 1.0, 'min_sum_hessian_in_leaf': 0.0835390405637034, 'num_leaves': 702.0, 'objective': 'regression'}


In [ ]:
# Trial 1
#18.405664646059204
#{'bagging_fraction': 0.9093160433687318, 'btype': 'I', 'extra_dims': 11, 'feature_fraction': 0.7968170824399049, 'lambda_l1': 0, 'lambda_l2': 0.00012297094946286044, 'learning_rate': 0.0390672755556049, 'metric': 'rmse', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 9.609043265244616e-07, 'num_leaves': 3.0, 'objective': 'regression'}

# Trial 2
#87.0875758035669
#{'bagging_fraction': 0.79782514300342, 'btype': 'R', 'extra_dims': 6, 'feature_fraction': 0.6228103392338451, 'lambda_l1': 5.797838307260306, 'lambda_l2': 0, 'learning_rate': 0.3232071142655213, 'metric': 'rmse', 'min_data_in_leaf': 160.0, 'min_sum_hessian_in_leaf': 0.11107148842459588, 'num_leaves': 53.0, 'objective': 'regression'}

# Trial 3
#21.322374028333776
#{'bagging_fraction': 0.896673102094286, 'btype': 'In', 'extra_dims': 3, 'feature_fraction': 0.6572158053747658, 'lambda_l1': 0.006965462022441443, 'lambda_l2': 0, 'learning_rate': 0.004568580120669364, 'metric': 'rmse', 'min_data_in_leaf': 110.0, 'min_sum_hessian_in_leaf': 2.3491196158122284e-07, 'num_leaves': 9.0, 'objective': 'regression'}

# Trial 4
#63.345702845570734
#{'bagging_fraction': 0.5735666948969093, 'btype': 'Rn', 'extra_dims': 5, 'feature_fraction': 0.8711919210303581, 'lambda_l1': 2.3052362030726136e-05, 'lambda_l2': 1.3913680033076565e-07, 'learning_rate': 0.21495392481070602, 'metric': 'rmse', 'min_data_in_leaf': 7.0, 'min_sum_hessian_in_leaf': 0.009732287287678531, 'num_leaves': 100.0, 'objective': 'regression'}

# Trial 5
#65.2899276851524
#{'bagging_fraction': 0.6717168979670439, 'btype': 'In', 'extra_dims': 10, 'feature_fraction': 0.7466331446375792, 'lambda_l1': 0, 'lambda_l2': 7.385966055147798, 'learning_rate': 0.9430705524333947, 'metric': 'rmse', 'min_data_in_leaf': 1.0, 'min_sum_hessian_in_leaf': 0.0835390405637034, 'num_leaves': 702.0, 'objective': 'regression'}